In [3]:
# Dependencies
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import zipfile
from pycocotools.coco import COCO
import os
import shutil
from PIL import Image
from tqdm.notebook import tqdm 
import json
import ipywidgets
import nbformat
import cv2
import functools

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import Callback


# Created models/functions for pre-processing data
from pre_processing import download_coco_datasets as dcd
from pre_processing import filter_coco_datasets as fcd
from pre_processing import resize_coco_datasets as rcd
from pre_processing import resize_annotations as ra

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
# Function to normalize pixel values to range [0,1]
def normalize_min_max(image):
    return image / 255.0

In [7]:
def load_annotations_from_json(json_file):
    with open(json_file, 'r') as f:
        annotations = json.load(f)
    
    # Assuming annotations is a list of items to iterate over
    for annotation in tqdm(annotations, desc="Loading Annotations"):
        # Replace process_annotation with your actual parsing logic
        # For example, print each annotation
        print(annotation)
    
    return annotations



In [13]:
def generate_image_and_annotation(image_dir, annotation_dir):
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg') or f.endswith('.jpeg')]
    annotations = load_annotations_from_json(os.path.join(annotation_dir, 'instances_train2017.json'))
    for filename in tqdm(image_files, desc="Loading Images"):
        img = cv2.imread(os.path.join(image_dir, filename))
        if img is None:
            continue
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        annotation_file = os.path.splitext(filename)[0] + '.json'
        annotation_path = os.path.join(annotation_dir, annotation_file)
        if not os.path.exists(annotation_path):
            continue
        with open(annotation_path, 'r') as f:
            annotation = json.load(f)
        yield img, annotation




In [14]:
# Path to pre-processed data directory
pre_processed_data = './coco_datasets/resized_images/'
annotations_dir = './coco_datasets/adjusted_annotations/'



# Load images and annotations using the generator
train_image_annotation_generator = generate_image_and_annotation(
    os.path.join(pre_processed_data, 'train2017_resized'), annotations_dir)
val_image_annotation_generator = generate_image_and_annotation(
    os.path.join(pre_processed_data, 'val2017_resized'), annotations_dir)
test_image_annotation_generator = generate_image_and_annotation(
    os.path.join(pre_processed_data, 'test2017_resized'), annotations_dir)

# Collect the generated data into lists
train_images, train_annotations = zip(*train_image_annotation_generator)
val_images, val_annotations = zip(*val_image_annotation_generator)
test_images, test_annotations = zip(*test_image_annotation_generator)

# Convert the lists to numpy arrays
train_images = np.array(train_images)
val_images = np.array(val_images)
test_images = np.array(test_images)

# Example usage:
for img, annotation in zip(train_images, train_annotations):
    # Process each training image and annotation here
    pass

for img, annotation in zip(val_images, val_annotations):
    # Process each validation image and annotation here
    pass

for img, annotation in zip(test_images, test_annotations):
    # Process each test image and annotation here
    pass

Loading Annotations:   0%|          | 0/5 [00:00<?, ?it/s]

info
licenses
images
annotations
categories


Loading Images:   0%|          | 0/118287 [00:00<?, ?it/s]

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
# 244x244 pixels, 3 channels (RGB)
input_shape = (224, 224, 3)
num_classes = 5  # xmin, ymin, xmax, ymax, class

# Initialize the model
model = Sequential()

# Add layers to the model
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes))  # Output layer

In [ ]:
# Compiling the model, using adam as optimizer and mean square error as loss function
model.compile(optimizer='adam', loss='mse') 

In [ ]:
model.summary()

In [ ]:
# Creating a log directory for visualization logs
# tensorboard --logdir logs/fit    # use in command to get url link for logs

log_dir = "logs/fit/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
class TQDMCallback(Callback):
    def __init__(self, total_epochs):
        super(TQDMCallback, self).__init__()
        self.total_epochs = total_epochs
        self.progress_bar = tqdm(total=self.total_epochs)

    def on_epoch_end(self, epoch, logs=None):
        self.progress_bar.update(1)
        self.progress_bar.set_description(f"Epoch {epoch+1}/{self.total_epochs}")
        self.progress_bar.set_postfix(logs)
        self.progress_bar.refresh()


In [ ]:
# Instantiate the TQDMCallback
tqdm_callback = TQDMCallback(total_epochs=10)



In [ ]:
batch_size = 32  # Define batch size

# Create data generators
train_generator = data_generator(train_image_loader, train_annotations, batch_size)
val_generator = data_generator(val_image_loader, val_annotations, batch_size)


# Calculate actual steps per epoch
steps_per_epoch = len(train_annotations) // batch_size

# Adjust validation steps
validation_steps = len(val_annotations) // batch_size




In [ ]:
# Iterate over the generator and print batches of data
for batch_images, batch_annotations in train_generator:
    print("Batch Images:", batch_images)
    print("Batch Annotations:", batch_annotations)


In [ ]:
# Training the model with 10 epochs and the data generators
model.fit(train_generator,
          steps_per_epoch=steps_per_epoch,
          validation_data=val_generator,
          validation_steps=validation_steps,
          epochs=10,
          callbacks=[tqdm_callback])

In [ ]:
# Evaluating the model on the test data
model.evaluate(test_data)

In [ ]:
# Saving the model
model.save('car_detector_model.h5')

In [ ]:
loaded_model = load_model('car_detector_model.h5')
predictions = loaded_model.predict(new_images) # REMOVE new_images for other data